# Импорты

In [269]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer
# from linear_regression_matrix import RidgeRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, roc_curve, r2_score, mean_squared_error

warnings.filterwarnings("ignore")

# Логистическая реализация

In [285]:
class LinearRegression(BaseEstimator):
    def __init__(self, learning_rate=0.1, max_epoches=1000, size_batch=60, eps=0.0000001):
        params = {
            "learning_rate" : learning_rate,
            "max_epoches" : max_epoches,
            "size_batch" : size_batch,
            "eps" : eps
        }

        for param_name, param_value in params.items():
            setattr(self, param_name, param_value)

        super(LinearRegression, self).set_params(**params)
        self.batches = []
        self.w = None
        self.diff_log = []


    def _stable_sigmoid(self, z: np.ndarray):
        """Sigmoid: функция преобразования предсказания модели в диапазон [0,1]"""
        z = np.sum(z)
        if z >= 0:
            return 1 / (1 + np.exp(-z))
        else:
            return np.exp(z) / (np.exp(z) + 1)


    def _log_loss(self, y_pred: np.ndarray, y: np.ndarray):
        """Логистическая функция потерь"""
        y_pred = self._stable_sigmoid(y_pred)
        y_one_loss = y * np.log(y_pred + 1e-9)
        y_zero_loss = (1 - y) * np.log(1 - y_pred + 1e-9)
        return -np.mean(y_zero_loss + y_one_loss)


    def _deriative_log_loss(self, X: np.ndarray, y_pred: np.ndarray, y: np.ndarray):
        """Производная лог лосса"""
        return np.dot(X.T, (y_pred - y)) / y_pred.shape[0]


    def _mse(self, y_pred: np.ndarray, y: np.ndarray):
        """MSE: функция отклонения таргета от предсказаний"""
        return np.divide(np.sum((y_pred - y)**2), len(y_pred))


    def _deriative_mse(self, X: np.ndarray, y_pred: np.ndarray, y: np.ndarray):
        """Градиента MSE"""
        return np.dot(np.divide(2*(y - y_pred), len(X)), (-X))


    def stable_softmax(X: np.ndarray, w: np.ndarray):
        """Функция SoftMax"""
        z = np.dot(-X, w)
        z = z - np.max(z, axis = -1, keepdims = True)
        numerator = np.exp(z)
        denominator = np.sum(numerator, axis = -1, keepdims = True)
        softmax = numerator / denominator
        return softmax


    def cross_entropy(y_pred: np.ndarray, y: np.ndarray, epsilon = 1e-9):
        """Кросс-энтропийная функция потерь в многоклассовой классификации"""
        n = y_pred.shape[0]
        ce = -np.sum(y * np.log(y_pred + epsilon)) / n
        return ce


    def gradient_softmax(self, X: np.ndarray, y_pred: np.ndarray, y: np.ndarray):
        """Градиент кросс-энтропийной функции потерь"""
        return np.array(1 / y_pred.shape[0] * np.dot(X.T, (y - y_pred)))


    def fit(self, X: np.ndarray, y: np.ndarray):
        """Обучение линейной регрессии на градиентном спуске: модификация Adam"""
        epoches = 0
        n_objects, n_features = X.shape
        self.w = np.random.normal(size=n_features)
        y_pred = X @ self.w
        while epoches < self.max_epoches:
            epoches += 1
            indices = np.random.choice(X.shape[0], size=self.size_batch, replace=False)
            # вычисление батчей
            X_batch = X[indices]
            y_batch = y[indices]
            # вычисление предсказаний
            y_pred = X_batch @ self.w
            # вычисление градиента
            grad = self._deriative_log_loss(X_batch, y_pred, y_batch)
            # oбновление весов
            self.w -= self.learning_rate * grad
            # проверка на то, что веса действительно изменились
            current_difference = self._log_loss(y_pred, y_batch)
            if current_difference < self.eps:
                break

        print(f"Count of epoches: {epoches}")

    def predict_proba(self, X_test):
        """Предсказываем вероятности соотнесения к объекту"""
        return X_test @ self.w

    def predict(self, X_test):
        """Предсказываем классы"""
        y_pred = self.predict_proba(X_test)
        y_pred = np.where(y_pred > 0.6, 1, -1)
        return y_pred


# Функции для отрисовки

In [270]:
def show_histplot(data: pd.DataFrame):
    data.hist(bins=20,figsize=(12,12))


def get_boxplot(df_column, column_name):
    pd.DataFrame(df_column).boxplot(sym='o', whis=1.0, showmeans=True)
    plt.show()


def get_3d(param1: list[int], param2: list[int], result: list[int], name_param1: str, name_param2: str):
    fig = plt.figure()
    ax = plt.axes(projection ='3d')
    ax.plot3D(param1, param2, result, 'green')
    ax.set_title(f'Зависимость метрики R² от {name_param1} и {name_param2}')
    plt.show()

def get_2d(param1: list[int], result: list[int], name_param1: str):
    plt.title(f'Зависимость метрики R² от {name_param1}')
    plt.plot(param1, result)

def plot_variance(pca, width=8, dpi=100):
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    fig.set(figwidth=8, dpi=100)
    return axs

def distribution_plots(h, w, X_train):
    fig3, ax3 = plt.subplots(h, w, sharex=True,figsize=(8, 8))
    axes_list = [item for sublist in ax3 for item in sublist]
    for col in X_train.columns:
        ax=axes_list.pop(0)
        sns.distplot(X_train[col], ax=ax)

    for ax in axes_list:
        ax.remove()

# Метрики классификации

In [271]:
def output_metrics_classification(y_test: pd.Series, preds: pd.Series):
    report = classification_report(y_test, preds, target_names=['Non-fail', 'Fail'])
    print(report)

def output_roc_auc(y_test: pd.Series, preds: pd.Series):
    sns.set(font_scale=1.5)
    sns.set_color_codes("muted")

    plt.figure(figsize=(5, 4))
    fpr, tpr, thresholds = roc_curve(y_test, preds, pos_label=1)
    lw = 2
    plt.plot(fpr, tpr, lw=lw, label='ROC curve ')
    plt.plot([0, 1], [0, 1])
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.savefig("ROC.png")
    plt.show()

# EDA: Influencers in Social Networks

In [272]:
data = pd.read_csv('train_recommend.csv')
X = data
y = X['Choice']
y[y == 0] = -1
X = X.drop('Choice', axis=1, inplace=False)
data.describe()

## Чекаем дупликаты

In [273]:
print(f"Number of missing value:{data.isna().sum().sum()}")

## Смотрим распределение

In [274]:
show_histplot(data)

## Смотрим на выбросы

### Ящик с усами

In [275]:
[get_boxplot(data[column], column) for column in data.columns]

### Смотрим на выбросы в процентах

In [276]:
def find_outliers(df):
    outliers = {}
    for col in df.columns:
        v = df[col]
        q1 = v.quantile(0.25)
        q3 = v.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers_count = ((v < lower_bound) | (v > upper_bound)).sum()
        perc = outliers_count * 100.0 / len(df)
        outliers[col] = (perc, outliers_count)
        print(f"Column {col} outliers = {perc:.2f}%")

    return outliers

outliers = find_outliers(data)

## Тепловая карта

In [277]:
sns.heatmap(data.corr(method='spearman'), vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')

**Вывод из тепловой карты:**

Такое ощущение, что B-признаки просто не влияют на таргет, но давайте посмотрим взаимную информацию, хотя интуитивно они должны влиять.

## Mutual Information

In [278]:
X['mean'] = X[X.columns].mean(axis=1)
X['std'] = X[X.columns].std(axis=1)
X['max'] = X[X.columns].max(axis=1)
X['median'] = X[X.columns].median(axis=1)

In [279]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

discrete_features = X.dtypes == int
mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[::3]
plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

**Итог из взаимной информации:**

Вывод из тепловой карты оказался неправильным, ведь как мы видим имеется взаимосвязь между таргетом и B-признаками.

А еще нет смысла добавлять mean/max/std/median.

# Feature Engineering

In [280]:
X = X.drop(['mean', 'max', 'median', 'std'], axis=1)

## Scaling

Можно по-разному масштабировать признаки:

1) логирование

2) MinMax

3) Standart scaling

In [281]:
def feature_transform(type_scaling, data):
    if type_scaling == 'standard':
        tran_fn = StandardScaler()
    elif type_scaling =='minmax':
        tran_fn = MinMaxScaler()
    elif type_scaling =='log':
        tran_fn = FunctionTransformer(np.log1p, validate=True)

    transfx_data = tran_fn.fit_transform(data.astype(float))
    transfx_data = pd.DataFrame(transfx_data, columns = data.columns)
    return transfx_data

X_minmax = feature_transform('minmax', X)
X_standard = feature_transform('standard', X)
X_log = feature_transform('log', X)

In [98]:
distribution_plots(4,3, X_log.iloc[:,:11])

In [99]:
distribution_plots(4,3, X_standard.iloc[:,:11])

In [100]:
distribution_plots(4,3, X_minmax.iloc[:,:11])

Поскольку признаки имеют ненормально распределение, то я бы использовала логарифмирование, к тому же с таким распределением легче будет прогнозировать.

## PCA

In [282]:
X_scaled = (X - X.mean(axis=0)) / X.std(axis=0)
pca = PCA()
X_pca = pca.fit_transform(X_scaled)
component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
X_pca = pd.DataFrame(X_pca, columns=component_names)

new_pca_df = pd.DataFrame(
    pca.components_.T,
    columns=component_names,
    index=X.columns,
)
plot_variance(pca)

После 16 компоненты примерно уровень объясненной дисперсии не особо меняется.

In [283]:
X_scaled = (X - X.mean(axis=0)) / X.std(axis=0)
pca = PCA(n_components=17)
X_pca = pca.fit_transform(X_scaled)
component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
X_pca = pd.DataFrame(X_pca, columns=component_names)

pca_df = pd.DataFrame(
    pca.components_.T,
    columns=component_names,
    index=X.columns,
)
X_pca

# Матричное решение

## Обучение на логарифмированных данных

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X_log, y, test_size=0.3, random_state=42)

rid_reg = RidgeRegressionMatrix(lambda_=0.1)
rid_reg.fit(X_train, y_train)
prob_predictions = rid_reg.predict_proba(X_test)
preds = rid_reg.predict(X_test)
output_metrics_classification(y_test, preds)
output_roc_auc(y_test, prob_predictions)

## Обучение на данных PCA

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

rid_reg = RidgeRegressionMatrix(lambda_=0.1)
rid_reg.fit(X_train, y_train)
prob_predictions = rid_reg.predict_proba(X_test)
preds = rid_reg.predict(X_test)
print(output_metrics_classification(y_test, preds))
output_roc_auc(y_test, prob_predictions)

## ИТОГ:

Логарифмирование дало лучший результат.

# Линейная регрессия

In [294]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

model_reg = LinearRegression(learning_rate=0.1, max_epoches=100, size_batch=100, eps=0.000000001)
model_reg.fit(X_train, y_train)

linreg_preds = model_reg.predict(X_test)
linreg_proba_preds = model_reg.predict_proba(X_test)

print(output_metrics_classification(y_test, linreg_preds))
output_roc_auc(y_test, linreg_proba_preds)

**Вывод:**

Результат мягко говоря не очень, но я точно уверена, что дело не в неправильной реализации линейной регрессии, так как я проверяла ту же реализацию на других датасетах (на Ирисе я получила Recall и Precision равными 1).

Нужно подумать, что здесь надо сделать